In [ ]:
import urllib3
import json
import certifi
import pandas as pd
import numpy as np
import time
import string
import openpyxl
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
def getRedditPosts(subReddit, loopCycle = 0):
    #loop due to reddit 100 post limit
    for i in range(loopCycle):
        #reddit only accepts one request every 2 seconds, adds a delay between each loop
        time.sleep(5)

        #set URL based on loop
        if i == 0:
            requestUrl = 'http://www.reddit.com/r/' + subReddit + '/.json?sort=top&t;=thisweek&limit;=100'
        else:    
            last = rawData[-1]['data']['name']
            requestUrl = 'https://www.reddit.com/r/' + subReddit + '/.json?sort=top&t;=thisweek&limit;=100&after;=%s' % last

        #actual payload get
        http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
        payload = http.request('GET', requestUrl)
        rawDataTemp = json.loads(payload.data.decode('utf-8'))

        #create initial json data or append
        if i == 0:
            rawData = rawDataTemp['data']['children']
        else:    
            rawData += rawDataTemp['data']['children']

        return rawData

In [ ]:
def cleanRedditData(rawData):
    tmpList = []
    dataNames = [value for value in rawData[0]['data']]
    for i in range(0, len(rawData)):
        dictionary = {}
        for names in dataNames:
            try:
                dictionary[str(names)] = rawData[i]['data'][str(names)]
            except:
                dictionary[str(names)] = 'None'
        tmpList.append(dictionary)
        
    return pd.DataFrame(tmpList)

In [ ]:
def tokenizeColumn(df,column):
    #prep string to tokenize
    df.drop(df.columns.difference([column]), 1, inplace=True)
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace('[^a-zA-Z]', ' ')
    df[column] = df[column].str.replace('[^\w\s]+', ' ')
    tokens = [word_tokenize(i) for i in df[column]]
    
    return tokens

In [ ]:
def normLanguage(tokens):
    #clean tokenized data
    df = pd.DataFrame(tokens)
    df = pd.melt(df)
    df.drop(columns=['variable'], inplace=True)
    df['value'] = df['value'].str.strip()
    df.dropna(inplace=True)

    # filter out stop words
    stopWords = set(stopwords.words('english'))
    wordList = [w for w in df['value'] if not w in stopWords]
    wordListDF = pd.DataFrame(wordList)
    
    return wordListDF

In [ ]:
rawJSON = getRedditPosts('cars', 10)
redditPostDF = cleanRedditData(rawJSON)
tokenizedTitles = tokenizeColumn(redditPostDF,'title')
wordList = normLanguage(tokenizedTitles)

wordList.to_excel('CleanWordList.xlsx', engine='openpyxl')